# Mortgage Calculator Program
### To generate a detailed mortgage payment breakout table
By Paul Zee-Cheng

December 15, 2021

In [1]:
def val_dec_err (func):
    '''
    To check if user input generates an error.
    '''
    def checking_wrapper(*args,**kwargs):
        while True:
            try:
                result = func(*args,**kwargs)

            except:
                print("There was a problem with your input, please try again.")

            else:
                return result
                
    return checking_wrapper

In [2]:
def val_dec_pos(func):
    '''
    To check that a user input is a positive number.
    '''
    def checking_wrapper(*args,**kwargs):
        while True:
            result = func(*args,**kwargs)
            
            if result >= 0:
                return result
            
            else:
                print("Amount must be at least zero!")
        
    return checking_wrapper

In [3]:
def val_dec_pct(func):
    '''
    To check that a user input is a percentage (integer between 0 to 100).
    '''
    def checking_wrapper(*args,**kwargs):
        while True:
            result = func(*args,**kwargs)
            
            if result >= 0 and result <= 100:
                return result
            
            else:
                print("Percentage must be from 0 to 100!")
    
    return checking_wrapper

In [4]:
#USER INPUT DEFINITIONS

@val_dec_pos
@val_dec_err
def property_fmv():
    '''
    To store property FMV (full purchase price). Store function as prop_fmv.
    '''
    return int(input("Enter purchase price of the property, rounded to the nearest dollar: "))

@val_dec_pct
@val_dec_err
def downpayment():
    '''
    To store downpayment percentage. Store as dp_rate.
    '''
    return float(input("Enter downpayment percentage, from 0 to 100: "))

@val_dec_pos
@val_dec_err
def loan_term():
    '''
    To store loan term (loan length). Store function as ln_term.
    '''
    return int(input("Enter the term of the loan, in full years (not partial years): "))

@val_dec_pct
@val_dec_err
def interest_rate():
    '''
    To store interest rate. Store as int_rate.
    '''
    return float(input("Enter interest rate percentage, from 0 to 100:"))

@val_dec_pos
@val_dec_err
def property_tax():
    '''
    To store property tax amount. Store function as ann_tax.
    '''
    return int(input("Enter the total annual property tax, rounded to the nearest dollar: "))

@val_dec_pos
@val_dec_err
def hoi_premium():
    '''
    To store homeowner's insurance premium. Store function as ann_hoi.
    '''
    return int(input("Enter the total annual homeowner's insurance premium, rounded to the nearest dollar: "))

@val_dec_pos
@val_dec_err
def pmi_premium(dp_rate):
    '''
    To store mortgage insurance premium. Requires dp_rate defined beforehand. Store function as ann_pmi.
    '''
    if dp_rate < 20:
        return int(input("Enter the total annual mortgage premium, rounded to the nearest dollar: "))
    else:
        print("Downpayment is at least 20%, no mortgage insurance required.")
        return 0


import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

@val_dec_err
def mortgage_start_date():
    '''
    To ask for mortgage start date. Required datetime & dateutil modules. Store as date0.
    '''
    date0 = input('Enter a date in YYYY-MM-DD format')
    year, month, day = map(int, date0.split('-'))
    return datetime.date(year,month,day)

In [5]:
def user_input_dictionary():
    '''
    To ask for all user inputs, and store them as a single dictionary. Store as ui_dc.
    '''
    ui_dc = {}
    
    ui_dc['prop_fmv'] = property_fmv()
    dp_rate_local = downpayment()
    ui_dc['dp_rate'] = dp_rate_local / 100
    ui_dc['ln_term'] = loan_term()
    ui_dc['int_rate'] = interest_rate() / 100
    ui_dc['ann_tax'] = property_tax()
    ui_dc['ann_hoi'] = hoi_premium()
    ui_dc['ann_pmi'] = pmi_premium(dp_rate_local) #user input differs from actual percent, must store separately
    ui_dc[ 'date0' ] = mortgage_start_date()
    
    return ui_dc

In [6]:
def derived_values_dictionary(ui_dc):
    '''
    To create a dictionary of the derived variables based on initial user inputs - easier to keep track
    of then putting everything into one giant dictionary. Store this as dr_dc.
    '''
    dr_dc = {}
    
    dr_dc['prin_amt'] = round(ui_dc['prop_fmv'] * ( 1 - ui_dc['dp_rate']  ), 2)
    dr_dc['mon_int'] = ui_dc['int_rate'] / 12
    dr_dc['num_pmt'] = ui_dc['ln_term'] * 12
    dr_dc['mon_tax'] = ui_dc['ann_tax'] / 12
    dr_dc['mon_hoi'] = ui_dc['ann_hoi'] / 12
    dr_dc['mon_pmi'] = ui_dc['ann_pmi'] / 12
    dr_dc['mon_tip'] = dr_dc['mon_tax'] + dr_dc['mon_hoi'] + dr_dc['mon_pmi'] #taxes, insurance, and HOI amount
    dr_dc['date0'] = ui_dc['date0'].strftime('%m-%d-%Y') #to convert datetime to string format
    
    return dr_dc

In [7]:
def monthly_pni_calc(dr_dc):
    '''
    To calculate monthly principle and interest payment amount, unchanged each month. Store variable as mon_pni.
    '''
    return dr_dc['prin_amt'] *  dr_dc['mon_int'] *  (  (1 + dr_dc['mon_int']) ** dr_dc['num_pmt'] ) / ( (1 +  dr_dc['mon_int']) ** dr_dc['num_pmt'] - 1 )    

In [8]:
def mortgage_table(ui_dc,dr_dc,mon_pi):
    
    zero_pmt = [0,0,0,dr_dc['prin_amt'],0,0,0,0,dr_dc['date0']]
            
    mr_list = [zero_pmt] #master list

    for pmt in range(1,dr_dc['num_pmt']+1):
    
        nw_pmt = [0,0,0,0,0,0,0,0,0]
        
        nw_pmt[0] = pmt #create a payment number
        
        new_interest_payment = mr_list[(pmt-1)][3] * dr_dc['mon_int'] #calculate interest payment
        if new_interest_payment < 0:
            nw_pmt[1] = 0
        else:
            nw_pmt[1] = round(new_interest_payment,2)
        
        new_principle_payment = mon_pi - nw_pmt[1] #calculate principle payment
        if mr_list[(pmt-1)][3] < mon_pi:
            nw_pmt[2] = round( ( mr_list[(pmt-1)][3] ) ,2)
        else:
            nw_pmt[2] = round(new_principle_payment,2)
        
        nw_pmt[3] = round( ( mr_list[(pmt-1)][3] - nw_pmt[2] ) , 2) #store new principle balance
        
        nw_pmt[4] = round( dr_dc['mon_tax'] , 2) #monthly taxes (same each month)
        
        nw_pmt[5] = round( dr_dc['mon_hoi'] , 2) #monthly insurance (same each month)
        
        nw_pmt[6] = round( dr_dc['mon_pmi'] , 2) #monthly mortgage insurance (same each month)
        
        nw_pmt[7] = round( nw_pmt[1] + nw_pmt[2] + nw_pmt[4] + nw_pmt[5] + nw_pmt[6] , 2)
        
        datetime_pmt = ui_dc['date0'] + relativedelta(months = pmt ) #actual date in datetime format
        nw_pmt[8] = datetime_pmt.strftime('%m-%d-%Y') #to convert datetime to string format
        
        mr_list.append(nw_pmt) #store the whole list in matrix
    
    return mr_list

In [9]:
def mortgage_calculator():
    '''
    Final calculator code, fully assembled.
    '''        
    print("Let's generate some mortgage tables! Enter in the terms of the loan to calculate.")
    input("Press any key to continue.")

    ui_dc = user_input_dictionary()
    dr_dc = derived_values_dictionary(ui_dc)
    mon_pi = monthly_pni_calc(dr_dc)
    mt = mortgage_table(ui_dc,dr_dc,mon_pi)
    
    ch = ['Payment #','Date','Balance','Principle','Interest','Tax,HOI,POI','Total'] #column header
    
    print('\n')
    print(f'{ch[0]:^11}|{ch[1]:^11}|{ch[2]:^11}|{ch[3]:^11}|{ch[4]:^11}|{ch[5]:^11}|{ch[6]:^11}')
    
    for row in mt:
        
        print(f'{row[0]:^11}|{row[8]:^11}|{row[3]:^11}|{row[2]:^11}|{row[1]:^11}|{(row[4]+row[5]+row[6]):^11}|{row[7]:^11}')
    

In [10]:
mortgage_calculator()

Let's generate some mortgage tables! Enter in the terms of the loan to calculate.


Press any key to continue. 
Enter purchase price of the property, rounded to the nearest dollar:  320000
Enter downpayment percentage, from 0 to 100:  20
Enter the term of the loan, in full years (not partial years):  30
Enter interest rate percentage, from 0 to 100: 4.0
Enter the total annual property tax, rounded to the nearest dollar:  8000
Enter the total annual homeowner's insurance premium, rounded to the nearest dollar:  3000


Downpayment is at least 20%, no mortgage insurance required.


Enter a date in YYYY-MM-DD format 2022-08-20




 Payment # |   Date    |  Balance  | Principle | Interest  |Tax,HOI,POI|   Total   
     0     |08-20-2022 | 256000.0  |     0     |     0     |     0     |     0     
     1     |09-20-2022 | 255631.15 |  368.85   |  853.33   |  916.67   |  2138.85  
     2     |10-20-2022 | 255261.07 |  370.08   |   852.1   |  916.67   |  2138.85  
     3     |11-20-2022 | 254889.76 |  371.31   |  850.87   |  916.67   |  2138.85  
     4     |12-20-2022 | 254517.21 |  372.55   |  849.63   |  916.67   |  2138.85  
     5     |01-20-2023 | 254143.42 |  373.79   |  848.39   |  916.67   |  2138.85  
     6     |02-20-2023 | 253768.38 |  375.04   |  847.14   |  916.67   |  2138.85  
     7     |03-20-2023 | 253392.09 |  376.29   |  845.89   |  916.67   |  2138.85  
     8     |04-20-2023 | 253014.55 |  377.54   |  844.64   |  916.67   |  2138.85  
     9     |05-20-2023 | 252635.75 |   378.8   |  843.38   |  916.67   |  2138.85  
    10     |06-20-2023 | 252255.69 |  380.06   |  842.12   |  916.67   |  